In [9]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
from refexps import get_previous
import pickle
from refexps import load_data, move_level, gamerun_level, get_previous #, lex_sim
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
import compound_splitter
from scipy import stats
from nltk import word_tokenize
from string import punctuation
punct = list(punctuation)
import compound_splitter

dataset_dir = '../En_De_Dataset/All/RefExp'
success_dir = '../En_De_Dataset/All/Success'

In [10]:
def exp_len(exp, language, splits, compound):
    '''
    Extract content words by the phrase
    :param exp: referring expression tuple
    :return: set of content words of the phrase
    '''
    exp = exp[2]
    words = word_tokenize(exp, language)
    if compound == True:
        if language == 'german':
            words_nocomp = []
            for w in words:
                word = compound_splitter.split_word(w, splits)
                words_nocomp = words_nocomp + (word.split(' '))	
                words = words_nocomp
    return len(words)

def len_div(exp1, exp2, language, splits, compound):
    '''
    compute dice similarity between the content words in two referring expression
    :param exp1: referring expression tuple 1
    :param exp2: referring expression tuple 2
    :return: dice similarity between the content words in the expressions; undefined if they have none
    '''        
    len1 = exp_len(exp1, language, splits, compound)    
    len2 = exp_len(exp2, language, splits, compound)
    if exp1 is 'no_previous' or exp2 is 'no_previous':
        return 'undefined'
    if len1 + len2 == 0:
        return 'undefined'
    metric = float(len1 - len2)/float(len1 + len2)
    return metric if not np.isnan(metric) and metric > -1  and metric < 1 else 'undefined'

In [13]:
def data_values_move(refexps_dir, success_dir, compound = True):
    splits = compound_splitter.load_dict('../de_lower.dict')
    move_level_data = defaultdict(dict)
    success_types = set()
    for dialogue in listdir(refexps_dir):
        #extract dialogue type
        success_file = dialogue.replace('refexps', 'movesuccess')
        success_file = path.join(success_dir, success_file)
        if dialogue.startswith('FTT'):
            language = 'german'
        else:
            language = 'english'
        dialogue_path = path.join(refexps_dir, dialogue)
        #load referring expressions data
        dialogue_path = refexps.load_data(dialogue_path)
        success_moves = refexps.load_data(success_file)
        #move level
        dialogue_move = refexps.move_level(dialogue_path)
        #iterate over moves
        for move in dialogue_move:
            move_id = dialogue + str(move[0])
            success = success_moves[move[0]]
            refs = move[1]
            if success == 'success':
                success = 'correct'
            if success == 'wrong':
                success = 'incorrect'
            if success == 'na':
                success = 'no_video'
            success_types.add(success)
            if success == 'no_video':
                pass
            else:
                refs = move[1]
                #iterate over objects
                for obj in refs:
                    #iterate over objects
                    refobj = refs[obj]
                    #SIMILARITY OF LEXICAL CONTENT INTO
                    sim_align = []
                    sim_align_speaker = defaultdict(list)
                    sim_coherence_speaker = defaultdict(list)
                    sim_coherence = []
                    for ref in refobj:
                        speaker = ref[1]
                        sim = len_div(ref, get_previous(ref, refobj), language, splits, compound) ###
                        if sim != 'undefined':
                            sim_align.append(sim)
                            sim_align_speaker[speaker].append(sim)
                    sim_align = np.mean(sim_align)
                    comb_ref = combinations(refobj,2)
                    for comb in comb_ref:
                        if comb[0] != comb[1]:
                            sim = len_div(comb[0], comb[1], language, splits, compound) ###
                            if sim != 'undefined':
                                sim_coherence.append(sim)
                                sim_coherence_speaker[comb[0][1]].append(sim)
                                sim_coherence_speaker[comb[1][1]].append(sim)
                    sim_coherence = np.mean(sim_coherence)
                    #movelevel coherence and alignment (for each object)
                    if not np.isnan(sim_coherence):
                        move_level_data['coherence'][move_id] = (sim_coherence, success)
                    if not np.isnan(sim_align):
                        move_level_data['alignment'][move_id] = (sim_align, success)
                    for s in sim_align_speaker:
                        if not np.isnan(np.mean(sim_align_speaker[s])):
                            move_level_data['alignment '+s.replace('-utts', '')][move_id] = (np.mean(sim_align_speaker[s]), success )
                    for s in sim_coherence_speaker:
                        if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                            move_level_data['coherence '+s.replace('-utts', '')][move_id] = (np.mean(sim_coherence_speaker[s]), success)
    return move_level_data

move_level_data = data_values_move(dataset_dir, success_dir)
data_regression = {}
for v in move_level_data:
    data_regression[v] = move_level_data[v].values()
pickle.dump(data_regression, open('regression_length.p', 'w'))

In [12]:
print data_regression

{'alignment e': [(-0.14285714285714285, 'correct'), (0.0, 'correct'), (0.0, 'correct'), (-0.5, 'not_moved'), (0.10000000000000001, 'not_moved'), (0.0, 'correct'), (0.5, 'correct'), (-0.33333333333333331, 'not_moved'), (-0.33333333333333331, 'not_moved'), (-0.875, 'incorrect'), (0.0, 'correct'), (-0.33333333333333331, 'correct'), (-0.69230769230769229, 'incorrect'), (0.20000000000000001, 'incorrect'), (0.0, 'not_moved'), (0.0, 'correct'), (0.0, 'correct'), (0.16666666666666666, 'correct'), (0.0, 'correct'), (-0.84615384615384615, 'not_moved'), (0.0, 'incorrect'), (0.25, 'incorrect'), (0.5, 'not_moved'), (0.33333333333333331, 'correct'), (0.14285714285714285, 'correct'), (0.5, 'correct'), (0.0, 'not_moved'), (0.34166666666666667, 'correct'), (0.33333333333333331, 'not_moved'), (-0.41666666666666669, 'correct'), (0.33333333333333331, 'not_moved'), (0.5, 'correct'), (0.0, 'not_moved'), (-0.23809523809523811, 'correct'), (0.0, 'correct'), (0.0, 'incorrect'), (0.63636363636363635, 'incorrect